In [16]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize
import os

In [10]:
save_path = r'C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_8\BA_mit_Git\BA_Plots\Probe_the_distance'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
# define some constants, theta gets treated as a variable throughout
# thetas are the theta values, which for the x-axis of my plots
fixed_g    = 0
fixed_dist = np.array([1, 1, 1])
fixed_phase_split = 1 / 3

start_gamma = -0.5
end_gamma   = 1
start_dist  = 0.3
end_dist    = 1.7
end_theta   = 2 * np.pi
end_time    = 10

############      arrays    ############
dist_size  = 50
split_size = 50
gamma_size = 50

# resolution:
theta_size  = 50
time_size   = 50

gammas = np.linspace(start_gamma, end_gamma, gamma_size)
splits = np.linspace(0, 1, split_size)

times  = np.linspace(0, end_time, time_size)
thetas = np.linspace(0, end_theta, theta_size)

In [11]:
#
#  Define the system
#
def phases(theta: float, split: float) -> tuple:
    """Computes the phase factors for a given angle.    0 <= split <= 1    """
    pa = np.exp(-1j * theta * split)
    pb = np.exp(-1j * theta * (1 - split) / 2)
    pc = np.exp(-1j * theta * (1 - split) / 2)
    
    return pa, pb, pc

def Hamilton(theta: float,
             gamma: float = fixed_g, 
             coeffs: np.array = fixed_dist, 
             split: float = fixed_phase_split) -> qt.Qobj:
    """Generates the Hermitian Hamiltonian matrix that creates the graph's topology.    """
    a, b, c    = coeffs
    pa, pb, pc = phases(theta, split)

    H = np.array([[0, 1, 0, 0, 0, 0],
                  [1, gamma, a * pa, b * np.conjugate(pb), 0, 0],
                  [0, a * np.conjugate(pa), gamma, c * pc, 1, 0],
                  [0, b * pb, c * np.conjugate(pc), gamma, 0, 1],
                  [0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0]], dtype=complex)
    
    return qt.Qobj(H)

basis1 = qt.basis(6, 0)  # |1> is the first basis vector
basis5 = qt.basis(6, 4)  # |5> is the fifth basis vector
basis6 = qt.basis(6, 5)  # |6> is the fifth basis vector

In [17]:
#
#  Calculate, save and plot the Probabilities
#
def Prob(times: np.array, theta: float, final_state: qt.Qobj, 
         coeffs = fixed_dist,
         gamma: float = fixed_g,
         split = fixed_phase_split):
    """    Calculate the probability of transitioning from an initial state to a final state
    over a given time, given the parameters of the Hamiltonian.    """
    H = Hamilton(theta, gamma, coeffs, split)
    output = qt.mesolve(H, basis1, times)
    evolved_states = output.states
    probability = np.array([np.abs((final_state.dag() * state))**2 for state in evolved_states])
    return probability

def P_arrays(times, thetas,
             distances = np.array([fixed_dist]),
             splits    = np.array([fixed_phase_split]),
             gammas    = np.array([fixed_g]),
             title = '',
             save = False,
             animate_dist = False, animate_phase = False, animate_gamma = False): # only one at a time
    P15 = np.zeros((len(gammas), len(distances), len(splits), len(thetas), len(times)), dtype=float)
    P16 = np.zeros((len(gammas), len(distances), len(splits), len(thetas), len(times)), dtype=float)
    save_word = ''
    
    if animate_dist:
        for c_idx, coeffs in enumerate(tqdm(distances, desc='Coefficients')):
            for split_idx, split in enumerate(splits):
                for g_idx, gamma in enumerate(gammas):
                    for theta_idx, theta in enumerate(thetas):
                        P15[g_idx, c_idx, split_idx, theta_idx, :] = Prob(times, theta, basis5, coeffs= coeffs, gamma=gamma, split=split)
                        P16[g_idx, c_idx, split_idx, theta_idx, :] = Prob(times, theta, basis6, coeffs= coeffs, gamma=gamma, split=split)

        for g_idx, gamma in enumerate(gammas):
            for split_idx, split in enumerate(splits):
                P15_min = P15[g_idx, :, split_idx, :, :].min()
                P15_max = P15[g_idx, :, split_idx, :, :].max()
                P15_norm = Normalize(vmin=P15_min, vmax=P15_max)
                filename_prefix = f'Ps_g={gammas[g_idx]:.2f}_split={splits[split_idx]:.2f}_' + title
                DIST_ani(P15[g_idx, :, split_idx, :, :], P16[g_idx, :, split_idx, :, :], filename_prefix, P15_norm, distances)
        if save:
            save_word = 'distance'

    if animate_phase:
        for split_idx, split in enumerate(tqdm(splits, desc='Phase Splitting')):
            for g_idx, gamma in enumerate(gammas):
                for c_idx, coeffs in enumerate(distances):
                    for theta_idx, theta in enumerate(thetas):
                        P15[g_idx, c_idx, split_idx, theta_idx, :] = Prob(times, theta, basis5, coeffs= coeffs, gamma=gamma, split=split)
                        P16[g_idx, c_idx, split_idx, theta_idx, :] = Prob(times, theta, basis6, coeffs= coeffs, gamma=gamma, split=split)

        for g_idx, gamma in enumerate(gammas):
            for c_idx, coeffs in enumerate(distances):
                P15_min = P15[g_idx, c_idx, :, :, :].min()
                P15_max = P15[g_idx, c_idx, :, :, :].max()
                P15_norm = Normalize(vmin=P15_min, vmax=P15_max)
                filename_prefix = f'Ps_g={gammas[g_idx]:.2f}_coeffs={distances[c_idx]}_' + title
                PHASE_ani(P15[g_idx, c_idx, :, :, :], P16[g_idx, c_idx, :, :, :], filename_prefix, P15_norm, splits)
        if save:
            save_word = 'phase_distribution'

        
    if animate_gamma:        
        for g_idx, gamma in enumerate(tqdm(gammas, desc='Gammas')):
            for c_idx, coeffs in enumerate(distances):
                for split_idx, split in enumerate(splits):
                    for theta_idx, theta in enumerate(thetas):
                        P15[g_idx, c_idx, split_idx, theta_idx, :] = Prob(times, theta, basis5, coeffs= coeffs, gamma=gamma, split=split)
                        P16[g_idx, c_idx, split_idx, theta_idx, :] = Prob(times, theta, basis6, coeffs= coeffs, gamma=gamma, split=split)        

        for c_idx, coeffs in enumerate(distances):
            for split_idx, split in enumerate(splits):
                P15_min = P15[:, c_idx, split_idx, :, :].min()
                P15_max = P15[:, c_idx, split_idx, :, :].max()
                P15_norm = Normalize(vmin=P15_min, vmax=P15_max)
                filename_prefix = f'Ps_coeffs={distances[c_idx]}_split={splits[split_idx]:.2f}_' + title
                GAMMA_ani(P15[:, c_idx, split_idx, :, :], P16[:, c_idx, split_idx, :, :], filename_prefix, P15_norm, gammas)
        if save:
            save_word = 'gamma'

    if save:
        np.save(save_path + f'/P15_values_with_' + title + '_Coeffs_animate_' + save_word +'.npy', P15)
        np.save(save_path + f'/P16_values_with_' + title + '_Coeffs_animate_' + save_word +'.npy', P16)

    return P15, P16

In [13]:
#
#  Plotting functions
#
def DIST_ani(P15_values, P16_values, filename_prefix, norm, distances):
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    # Initial plot to set up the colorbar
    im1 = axs[0].imshow(P15_values[0], extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                   aspect='auto', origin='lower', cmap='viridis', norm=norm)
    im2 = axs[1].imshow(P16_values[0], extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                   aspect='auto', origin='lower', cmap='viridis', norm=norm)
    cbar = fig.colorbar(im2, ax=axs[1])
    cbar.set_label('P Value')


    def update(frame):
        for ax in axs:
            ax.clear()
        
        P15_frame = P15_values[frame, :, :].T
        P16_frame = P16_values[frame, :, :].T

        axs[0].set_title(f'P15: coefficients = [{distances[frame][0]:.2f}, {distances[frame][1]:.2f}, {distances[frame][2]:.2f}]')
        axs[0].set_xlabel("Phase θ")
        axs[0].set_ylabel('Time t')
        axs[1].set_title(f'P16')

        im1 = axs[0].imshow(P15_frame, extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                            aspect='auto', origin='lower', cmap='viridis', norm=norm)
        im2 = axs[1].imshow(P16_frame, extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                            aspect='auto', origin='lower', cmap='viridis', norm=norm)
        cbar.update_normal(im2)  # Update the colorbar
        
    ani = FuncAnimation(fig, update, frames=len(distances), interval=200)
    ani.save(save_path + f'/DistDep/{filename_prefix}.gif', writer='pillow', fps=10)
    plt.close(fig)

def PHASE_ani(P15_values, P16_values, filename_prefix, norm, splits):
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    # Initial plot to set up the colorbar
    im1 = axs[0].imshow(P15_values[0], extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                   aspect='auto', origin='lower', cmap='viridis', norm=norm)
    im2 = axs[1].imshow(P16_values[0], extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                   aspect='auto', origin='lower', cmap='viridis', norm=norm)
    cbar = fig.colorbar(im2, ax=axs[1])
    cbar.set_label('P Value')

    def update(frame):
        for ax in axs:
            ax.clear()
        
        P15_frame = P15_values[frame, :, :].T
        P16_frame = P16_values[frame, :, :].T

        axs[0].set_title(f'P15: Phase split = {splits[frame]:.2f}')
        axs[0].set_xlabel("Phase θ")
        axs[0].set_ylabel('Time t')
        axs[1].set_title(f'P16')

        im1 = axs[0].imshow(P15_frame, extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                            aspect='auto', origin='lower', cmap='viridis', norm=norm)
        im2 = axs[1].imshow(P16_frame, extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                            aspect='auto', origin='lower', cmap='viridis', norm=norm)

        cbar.update_normal(im2)  # Update the colorbar
        
    ani = FuncAnimation(fig, update, frames=len(splits), interval=200)
    ani.save(save_path + f'/PhaseDep/{filename_prefix}.gif', writer='pillow', fps=10)
    plt.close(fig)
    
def GAMMA_ani(P15_values, P16_values, filename_prefix, norm, gammas):
    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    
    # Initial plot to set up the colorbar
    im1 = axs[0].imshow(P15_values[0], extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                   aspect='auto', origin='lower', cmap='viridis', norm=norm)
    im2 = axs[1].imshow(P16_values[0], extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                   aspect='auto', origin='lower', cmap='viridis', norm=norm)
    cbar = fig.colorbar(im2, ax=axs[1])
    cbar.set_label('P Value')

    def update(frame):
        for ax in axs:
            ax.clear()
        
        P15_frame = P15_values[frame, :, :].T
        P16_frame = P16_values[frame, :, :].T

        axs[0].set_title(f'P15: gamma = {gammas[frame]:.2f}')
        axs[0].set_xlabel("Phase θ")
        axs[0].set_ylabel('Time t')
        axs[1].set_title(f'P16')

        im1 = axs[0].imshow(P15_frame, extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                            aspect='auto', origin='lower', cmap='viridis', norm=norm)
        im2 = axs[1].imshow(P16_frame, extent=[thetas.min(), thetas.max(), times.min(), times.max()],
                            aspect='auto', origin='lower', cmap='viridis', norm=norm)
        cbar.update_normal(im2)  # Update the colorbar
        
    ani = FuncAnimation(fig, update, frames=len(gammas), interval=200)
    ani.save(save_path + f'/GammaDep/{filename_prefix}.gif', writer='pillow', fps=10)
    plt.close(fig)

### Symmetric case:

In [14]:
coeffs = np.array([[coeff, coeff, coeff] for coeff in np.linspace(start_dist, end_dist, dist_size)])
title = 'symmetric'
#np.save(f'Animation/' + title + '_distances.npy', coeffs)

In [15]:
# Define the parameters for the animations
ex_dists = np.array([[0.2, 0.2, 0.2], [1, 1, 1]])
ex_splits = np.array([0, 1/3, 1])
ex_gammas = np.array([-0.5, 1])
#
#  Animate the Distance for gammas = -0.5, 0, 1, , splits = 0, 1/3, 1
#
P_arrays(times, thetas, distances=coeffs, gammas=ex_gammas, title=title, save=False, animate_dist=True)
#
#  Animate the Phase distr. for coeffs = [0.2, 0.2, 0.2], [1, 1, 1], gammas = -0.5, 0, 1 -> IT HAS NO EFFECT!!
#
#P_arrays(times, thetas, distances=ex_dists, gammas=ex_gammas, splits=splits, title=title, save=False, animate_phase=True)
#
#  Animate the Gamma for  coeffs = [0.2, 0.2, 0.2], [1, 1, 1], splits = 0, 1/3, 1
#
P_arrays(times, thetas, distances=ex_dists, gammas=gammas, splits=ex_splits, title=title, animate_gamma=True) # save=True,

### Asymmetric Case

In [ ]:
ex_dists = np.array([[0.2, 0.3, 0.4]])
splits_phase = np.array([0, 1])
ex_gammas = np.array([-0.5, 1])
#
#  Animate the Distance for gammas = -0.5, 0, 1, , splits = 0, 1/3, 1
#
#P_arrays(times, thetas, distances=coeffs, gammas=ex_gammas, title=title, save=True, animate_dist=True)

#
#  Animate the Phase distr. for coeffs = [0.2, 0.2, 0.2], [1, 1, 1], gammas = -0.5, 0, 1 -> IT HAS NO EFFECT!!
#
#P_arrays(times, thetas, distances=ex_dists, gammas=ex_gammas, splits=splits, title=title, save=True, animate_phase=True)
#
#  Animate the Gamma for  coeffs = [0.2, 0.2, 0.2], [1, 1, 1], splits = 0, 1/3, 1
#
#P_arrays(times, thetas, distances=ex_dists, gammas=gammas, splits=splits_phase, title=title, save=True, animate_gamma=True)

In [ ]:
mid_dist = (start_dist + end_dist) / 2
if dist_size != 1:
    increment = (end_dist - start_dist) / (dist_size - 1)
else:
    increment = 0.1
coeffs = np.array([[mid_dist, mid_dist, start_dist + i * increment] for i in range(dist_size)])
title = 'a-symmetric'
#np.save(title + '_distances.npy', coeffs)